# Primeros Pasos con APIs: Ejemplo Agronómico Básico

## Configuración de la API de OpenAI

**¿Qué es una API?** Una Interfaz de Programación de Aplicaciones que nos permite acceder a servicios de inteligencia artificial desde nuestros programas Python. En agricultura, esto nos permite automatizar análisis de datos de campo, reportes, y toma de decisiones.

In [ ]:
!pip install --upgrade openai dotenv --quiet

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()  # Cargar variables de entorno desde un archivo .env si existe

# Intentar detectar Colab
IN_COLAB = False
try:
    import google.colab
    from google.colab import userdata
    IN_COLAB = True
except Exception:
    IN_COLAB = False

OPENAI_API_KEY = None
if IN_COLAB:
    # En Colab el usuario puede guardar secretos y recuperarlos con userdata.get
    try:
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    except Exception:
        OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
else:
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not OPENAI_API_KEY:
    raise ValueError('OPENAI_API_KEY no encontrada. Coloca tu clave en .env o en Colab secrets con la key OPENAI_API_KEY')

print(f"✅ OPENAI key cargada. Entorno: {'Colab' if IN_COLAB else 'Local'}")

# Tomar de acá: https://platform.openai.com/account/api-keys

## Función básica para consultas agrícolas

Esta función nos permite enviar cualquier consulta agronómica a ChatGPT y recibir una respuesta estructurada. Es la base para automatizar análisis de datos de campo.

In [ ]:
def get_completion(prompt, model="gpt-4o-mini", temperature=0):
    # Usamos la variable OPENAI_API_KEY cargada desde secrets o .env
    client = OpenAI(api_key=OPENAI_API_KEY)

    messages = [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature, # grado de aleatoriedad (0 = más predecible)
    )
    return response.choices[0].message.content

## Primera consulta agronómica: Análisis de resultados de laboratorio

In [ ]:
analisis_laboratorio = """
ANÁLISIS DE SUELO - LABORATORIO AGROPAMPA
Cliente: Establecimiento San José - Pergamino, Buenos Aires
Lote: 12-A (Preparación para soja 2024/25)
Fecha: 08/01/2024
Profundidad: 0-20 cm

RESULTADOS:
pH: 5.2 (ácido)
Materia orgánica: 2.8% (medio)
Nitrógeno total: 0.12% (bajo)
Fósforo disponible: 12 ppm (bajo)
Potasio intercambiable: 180 ppm (medio)
Conductividad eléctrica: 0.4 dS/m (normal)
Textura: Franco-limoso
CIC: 22.5 cmol/kg

OBSERVACIONES: Suelo típico de la pampa húmeda con acidez moderada.
Recomiendan encalado previo a siembra. Evaluar fertilización fosfatada
según análisis económico de dosis respuesta.
"""

In [ ]:
prompt = f"""
Analiza los siguientes resultados de análisis de suelo y extrae la información clave:
- Estado general del suelo (bueno/regular/deficiente)
- ¿Requiere encalado? (verdadero o falso)
- Nutriente más limitante
- Establecimiento/Cliente

Formatea tu respuesta como un objeto JSON con las claves:
"estado_suelo", "requiere_encalado", "nutriente_limitante" y "establecimiento".

Si la información no está presente, usa "no_especificado" como valor.
Formatea el valor de requiere_encalado como un booleano.

Análisis de laboratorio: '''{analisis_laboratorio}'''
"""
response = get_completion(prompt)
print(response)

## Ejemplo práctico: Generación de recomendaciones

Una vez que dominamos lo básico, podemos usar la API para generar recomendaciones técnicas automatizadas.

In [ ]:
# Generamos recomendaciones técnicas automáticamente
prompt_recomendaciones = f"""
Basado en este análisis de suelo, genera 3 recomendaciones técnicas específicas
para la siembra de soja en la región pampeana:

{analisis_laboratorio}

Incluye:
1. Corrección de pH (dosis de cal si corresponde)
2. Plan de fertilización (productos y dosis por hectárea)
3. Manejo específico del lote

Responde de forma técnica pero práctica.
"""

recomendaciones = get_completion(prompt_recomendaciones)
print("=== RECOMENDACIONES TÉCNICAS ===")
print(recomendaciones)

In [ ]:
from IPython.display import Markdown, display

# Imprimir la respuesta en formato Markdown
display(Markdown(recomendaciones))

## ¿Qué aprendimos?

1. **¿Qué es una API?** Una forma de comunicarnos con servicios de inteligencia artificial
2. **Configuración básica**: Cómo cargar credenciales de OpenAI de forma segura
3. **Función de consulta**: `get_completion()` nos permite hacer cualquier pregunta
4. **Casos de uso reales**: Análisis de laboratorio, extracción de datos, recomendaciones técnicas

### Próximo paso:
En el siguiente cuaderno veremos cómo usar **Google Gemini API** para diversificar nuestras herramientas de IA agrícola.